In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

# 데이터 로딩

In [ ]:
trainset_raw = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
testset_raw = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

In [ ]:
def preprocess_cifar(dataset):
    images = torch.tensor(np.transpose(np.array(dataset.data), (0,3,1,2)), dtype=torch.float32) / 255.0
    mean = images.mean([0,2,3], keepdim=True)
    std = images.std([0,2,3], keepdim=True)
    images = (images - mean) / std
    labels = torch.tensor(dataset.targets)
    return images, labels

In [ ]:
train_images, train_labels = preprocess_cifar(trainset_raw)
test_images, test_labels = preprocess_cifar(testset_raw)

In [ ]:
# train/val split
train_size = int(0.8 * len(train_images))
val_size = len(train_images) - train_size

In [ ]:
train_images, val_images = torch.utils.data.random_split(train_images, [train_size, val_size])
train_labels, val_labels = torch.utils.data.random_split(train_labels, [train_size, val_size])

In [ ]:
trainloader = torch.utils.data.DataLoader(list(zip(train_images, train_labels)), batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(list(zip(val_images, val_labels)), batch_size=64, shuffle=False)

# 모델 생성

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습

In [ ]:
train_losses, val_losses, val_accuracies = [], [], []

In [ ]:
for epoch in range(20):
    model.train()
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(trainloader))

    # ----- Validation -----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in valloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_losses.append(val_loss / len(valloader))
    val_accuracies.append(100 * correct / total)

    print(f"Epoch [{epoch+1}/20], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.2f}%")


# 학습결과 확인

In [ ]:
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
plt.figure()
plt.plot(val_accuracies, label="Val Accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()